In [0]:
!curl "https://storage.googleapis.com/kagglesdsdata/competitions/6277/323734/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1586094727&Signature=QqJTGqDDnLO5fFXc74BxM5Txqh3njzGWrlJsh"%"2FlBFAeqIrjcq9LtrbkPx2zsoGkIQfZp56ffDGuJ"%"2BvAUYri"%"2FhZ3zyEYjBN67xiOAV"%"2FY3Y2l8Qfufhu0HC1MdNivKXavBwBL9lqJmVwfNc"%"2F5e94x7e"%"2FgYflVlNv5ndFwyNoflt5z5sa"%"2FDp"%"2FDpNpuyv38zT985jt5yMUD9VotwApx4EHtldPprE7qQnyMmGNtqUEM39bw4043ViMn"%"2BPT79XMLlukXYt"%"2FpC"%"2FjTURdaYA6eEBnIG5"%"2FPGLNFG"%"2BbkStGx71ZwkQ7n3iGzLKFLofldHYgGIfTpOkZnRj"%"2FN8GCaEQvesX566Zw"%"3D"%"3D&response-content-disposition=attachment"%"3B+filename"%"3Dtrain.csv.zip" -H "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0" -H "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8" -H "Accept-Language: en-US,en;q=0.5" --compressed -H "Referer: https://www.kaggle.com/" -H "Connection: keep-alive" -H "Upgrade-Insecure-Requests: 1" -o train.csv.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.1M  100 21.1M    0     0  22.8M      0 --:--:-- --:--:-- --:--:-- 22.8M


In [0]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
import re
import csv
import codecs

In [0]:
df  = pd.read_csv("train.csv")

In [0]:
df.tail()

,id,qid1,qid2,question1,question2,is_duplicate
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0
404289,404289,537932,537933,What is like to have sex with cousin?,What is it like to have sex with your cousin?,0


In [0]:
new_df = df.iloc[:2000, :]

In [0]:
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler, Dataset

In [0]:
import unicodedata
import string
import numpy as np

In [0]:
# pytorch tutorial
class Word:

  def __init__(self):

    self.word2index = {'<blank>':0}
    self.word2count = {}
    self.index2word = {0:'<blank>'}
    self.n_words = 1

  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)

  def addWord(self, word):
    if word not in self.word2index:
      
      self.word2index[word] = self.n_words
      self.word2count[word] = 1
      self.index2word[self.n_words] = word
      self.n_words +=1

    else:
      self.word2count[word] += 1

In [0]:
word = Word()

In [0]:
d = new_df["question1"][:20]

In [0]:
d[9]

'Motorola (company): Can I hack my Charter Motorolla DCX3400?'

In [0]:
# https://www.kaggle.com/currie32/quora-question-pairs/the-importance-of-cleaning-text
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [0]:
text_to_wordlist(d[9])

'motorola company : can i hack my charter motorolla dcx3400 '

In [0]:
concat_q1q1 = pd.concat([new_df['question1'],new_df['question2']],0)

In [0]:
clean_text = list(map(lambda x: text_to_wordlist(x),concat_q1q1.to_list()))

BUild word lang from the clean text

In [0]:
word = Word()

In [0]:
for text in clean_text:
  word.addSentence(text)

In [0]:
def tokenize(lang,sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

In [0]:
question_tok = list(map(lambda x: tokenize(word,x), clean_text))

In [0]:
question_tok[700]

[22, 71, 72, 1040, 31, 150, 1042, 131, 2259, 2260, 13]

In [0]:
list_len = list(map(lambda x: len(x), question_tok))

In [0]:
MAX_LENGTH = np.array(list_len).max()

In [0]:
MAX_LENGTH

68

PAD with Zero

In [0]:
pad_text = np.zeros((len(question_tok), MAX_LENGTH))

In [0]:
for i,v in enumerate(question_tok):

  for j, token in enumerate(v):
    
    pad_text[i,j] = token

In [0]:
ques1 = pad_text[:2000]
ques2 = pad_text[2000:]

In [0]:
import torch

In [0]:
class Ques(Dataset):

  def __init__(self,q1, q2, label,**init_kwargs):

    self.q1 = torch.from_numpy(q1).long()
    self.q2 = torch.from_numpy(q2).long()
    self.label = torch.Tensor(label.to_list()).float()

  def __len__(self):
    return len(self.label)

  def __getitem__(self, idx):
    x1 = self.q1[idx]
    x2 = self.q2[idx]
    label = self.label[idx]

    return (x1, x2), label

In [0]:
ques = Ques(ques1, ques2,new_df['is_duplicate'])

In [0]:
ques[100]

((tensor([222, 543, 544,  67, 465, 545, 293,  12,  48, 546, 547,   3, 548, 549,
            1, 222,  67, 299, 482,  13,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]),
  tensor([ 222,  543,   67,   31, 3965,  545,  293,   12,   48,  546,   13,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0])),
 tensor(1.))

In [0]:
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
valid_sz = 0.1

In [0]:
num_train = len(new_df)
indices = list(range(num_train))
split = int(np.floor(valid_sz * num_train))

In [0]:
split

200

In [0]:
shuffle = True

if shuffle:
  np.random.shuffle(indices)

In [0]:
train_idx, valid_idx= indices[split:], indices[:split]

In [0]:
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [0]:
bs = 20

In [0]:
train_dl = DataLoader(ques,batch_size=bs, sampler=train_sampler)
valid_dl = DataLoader(ques,batch_size=bs, sampler=valid_sampler)

In [0]:
(x1, x2), y = next(iter(train_dl))

In [0]:
x1.shape

torch.Size([20, 68])

In [0]:
from torchtext import vocab

In [0]:
golve = vocab.GloVe(name='6B', dim=300)

.vector_cache/glove.6B.zip: 862MB [06:28, 2.22MB/s]                          
100%|█████████▉| 399448/400000 [00:36<00:00, 10919.48it/s]

In [0]:
weights_matrix = np.zeros((matrix_len, 50))
words_found = 0

for i, word in enumerate(target_vocab):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

In [0]:
word.n_words

6285

In [0]:
weights_matrix = np.zeros((word.n_words, 300))
words_found = 0

for i , wd in enumerate(list(word.word2index.keys())):
  try:
    weights_matrix[i] = golve[wd]
    words_found +=1
  except KeyError:
    weights_matrix[i] = np.random.normal(scale=0.6, size=(300, ))

In [0]:
weights_matrix = torch.from_numpy(weight_matrix).float()

In [0]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.shape
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

In [0]:
import torch.nn.functional as F
from torch import nn

In [0]:
class MVLSTM(nn.Module):

  def __init__(self,weights_matrix, hidden_dim,hid_out, Max_length, n_class,topk=10):

    super().__init__()

    self.embed, _, embed_dim = create_emb_layer(weights_matrix)

    self.bilstm1 = nn.LSTM(embed_dim,hidden_dim, batch_first=True, bidirectional=True)
    self.bilstm2 = nn.LSTM(embed_dim,hidden_dim, batch_first=True, bidirectional=True)

    self.linear = nn.Linear(Max_length * topk, hid_out)
    self.linear2 = nn.Linear(hid_out, n_class)

    self.max_length = Max_length
    self.topk = topk

  def forward(self,x1,x2):

    embed1 = self.embed(x1)
    embed2 = self.embed(x2)

    q1, _ = self.bilstm1(embed1)
    q2, _ = self.bilstm2(embed2)

    q2 = q2.permute(0,2,1)

    dot = torch.bmm(q1,q2)

    topk = dot.topk(self.topk).values.view(x1.shape[0], -1)

    lin = F.relu(self.linear(topk))
    output = self.linear2(lin)

    return output

In [0]:
cv = MVLSTM(weights_matrix,256,20,MAX_LENGTH,2)

In [0]:
p = cv(x1,x2)

In [0]:
p.shape

torch.Size([20, 2])

In [0]:
torch.optim.

In [0]:
lr = 0.1

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cv.parameters(), lr=lr)

In [0]:
batch_size=20

In [0]:
epochs = 20

counter = 0
print_every = 100
clip = 2.5

cv.cuda()

cv.train()

train_on_gpu = True

for e in range(epochs):

    print(f"Epoch {e} begins")
    # batch loop
    for inputs, labels in train_dl:
        counter += 1

        if(train_on_gpu):
            inp1, inp2, labels = inputs[0].cuda(),inputs[1].cuda(), labels.cuda()

        # zero accumulated gradients
        optimizer.zero_grad()


        # get the output from the model
        output= cv(inp1,inp2)
        # print("train output",output[0])
        # calculate the loss and perform backprop
        # print(labels,output.shape)
        loss = criterion(output, labels.long())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(cv.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_losses = []
            cv.eval()
            for inputs, labels in valid_dl:

                if(train_on_gpu):
                    inp1, inp2, labels = inputs[0].cuda(),inputs[1].cuda(), labels.cuda()
              
                output= cv(inp1,inp2)
                # print("validation output",output)
                val_loss = criterion(output, labels.long())

                val_losses.append(val_loss.item())

            cv.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch 0 begins
Epoch 1 begins
Epoch: 2/20... Step: 100... Loss: 0.642891... Val Loss: 0.659608
Epoch 2 begins
Epoch: 3/20... Step: 200... Loss: 0.571375... Val Loss: 0.650537
Epoch 3 begins
Epoch: 4/20... Step: 300... Loss: 0.539332... Val Loss: 0.656195
Epoch 4 begins
Epoch: 5/20... Step: 400... Loss: 0.627502... Val Loss: 0.652030
Epoch 5 begins
Epoch: 6/20... Step: 500... Loss: 0.659280... Val Loss: 0.664409
Epoch 6 begins
Epoch: 7/20... Step: 600... Loss: 0.647753... Val Loss: 0.650625
Epoch 7 begins
Epoch: 8/20... Step: 700... Loss: 0.631781... Val Loss: 0.662258
Epoch 8 begins
Epoch: 9/20... Step: 800... Loss: 0.858816... Val Loss: 0.651047
Epoch 9 begins
Epoch: 10/20... Step: 900... Loss: 0.675997... Val Loss: 0.651479
Epoch 10 begins
Epoch 11 begins
Epoch: 12/20... Step: 1000... Loss: 0.647846... Val Loss: 0.656586
Epoch 12 begins
Epoch: 13/20... Step: 1100... Loss: 0.649909... Val Loss: 0.652999
Epoch 13 begins
Epoch: 14/20... Step: 1200... Loss: 0.649927... Val Loss: 0.655502